In [ ]:
import os, time, random, logging , datetime , cv2 , csv , subprocess , json
import numpy as np


import tensorflow as tf
print("tf",tf.version.VERSION)
from tensorflow import keras


import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.cm import get_cmap
from scipy.interpolate import griddata

import plotly.express as px
import plotly.graph_objs as go
import plotly.io as pio
import plotly.subplots as sp
import pandas as pd


from utils import globo , xdv , tfh5 , sinet


''' GPU CONFIGURATION '''

#os.environ["CUDA_VISIBLE_DEVICES"]="1,2"
#tfh5.set_tf_loglevel(logging.ERROR)
#tfh5.tf.debugging.set_log_device_placement(False) #Enabling device placement logging causes any Tensor allocations or operations to be printed.
#tfh5.set_memory_growth()
#os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'


CFG_SINET = {
    
    'sinet_version': 'fsd-sinet-vgg42-tlpf_aps-1',
    
    'graph_filename' : os.path.join(globo.FSDSINET_PATH,"fsd-sinet-vgg42-tlpf_aps-1.pb"),
    'metadata_file'  : os.path.join(globo.FSDSINET_PATH,"fsd-sinet-vgg42-tlpf_aps-1.json"),
    
    'audio_fs_input':22050,
    'batchSize' : 64,
    'lastPatchMode': 'repeat',
    'patchHopSize' : 50,
    
    
    'anom_labels' : ["Alarm","Boom","Crowd","Dog","Drill","Explosion","Fire","Gunshot and gunfire","Hammer","Screaming","Screech",\
                    "Shatter","Shout","Siren","Slam","Squeak","Yell"],
    'anom_labels_i' : [4,18,51,59,65,72,78,92,94,145,146,147,148,152,154,161,198],
    
    'anom_labels2' : ["Boom","Explosion","Fire","Gunshot and gunfire","Screaming",\
                    "Shout","Siren","Yell"],
    'anom_labels_i2' : [18,72,78,92,147,148,152,198],
    
    'full_or_max' : 'max', #chose output to (timesteps,labels_total) ot (1,labels_total)
    'labels_total' : 200
    
}
CFG_WAV= {
    
    "full_or_max" : CFG_SINET["full_or_max"],
    "sinet_aas_len" : CFG_SINET["labels_total"], 
    
    "shuffle" : False,
    
    "ativa" : 'relu',
    "optima" : 'sgd',
    "batch_type" : 0,   # =0 all batch have frame_max or video length // =1 last batch has frame_max frames // =2 last batch has no repetead frames
    "frame_max" : 8000,
    "ckpt_start" : f"{0:0>8}",  #used in train_model: if 00000000 start from scratch, else start from ckpt with CFG_WAV stated
    
    "epochs" : 1,
    "batch_size" : 1
    
}

In [ ]:
data = np.load(os.path.join(globo.AAS_PATH,f"{CFG_SINET['sinet_version']}--fl_train.npz"), allow_pickle=True)["data"]

p_es_arr_total = ([data[k]['p_es_array'] for k in range(len(data))])
labels_total = [data[k]['label'] for k in range(len(data))]

# Convert the lists to NumPy arrays
p_es_arr_total = np.array(p_es_arr_total)
labels_total = np.array(labels_total)

# Check the shapes of the resulting NumPy arrays
print("Shape of p_es_arr_total_np:", np.shape(p_es_arr_total[1])[1])
print("Shape of labels_total_np:", labels_total.shape)

In [ ]:
'''
def pad_arrays(arrays, pad_value=0):
    max_shape = np.max([arr.shape for arr in arrays], axis=0)
    padded_arrays = []

    for arr in arrays:
        pad_width = [(0, max_dim - dim) for dim, max_dim in zip(arr.shape, max_shape)]
        padded_array = np.pad(arr, pad_width, mode='constant', constant_values=pad_value)
        padded_arrays.append(padded_array)

    return np.stack(padded_arrays)
'''

#p_es_arr_total = pad_arrays(p_es_arr_total)

In [ ]:
def get_them_stats(total_array,total_labels):
    nintervals = np.shape(total_array)[0] #, _, nclasses
    nclasses = total_array[1].shape[1]
    
    maxx, avgg, meantopkmax = [None] * nintervals, [None] * nintervals, [None] * nintervals 
    k = 10
    for i in range(nintervals):
        #print(i,np.shape(total_array[i]))
        # 1. Max probability
        maxx[i] = np.max(total_array[i], axis=0)

        # 3. top-interval max
        # Sort the values along the time_steps axis (axis=0) for each class in descending order
        sorted_values_per_class = np.sort(total_array[i], axis=0)[::-1]
        # Select the top-k values for each class
        top_k_values_per_class = sorted_values_per_class[:k, :]
        # Calculate the mean of the top-k values for each class
        meantopkmax[i] = np.mean(top_k_values_per_class, axis=0)
        
                
        # 2. Average probability
        avgg[i] = np.mean(total_array[i], axis=0)


    # 9. Autocorrelation
    #def autocorr(x):
    #    result = np.correlate(x, x, mode='full')
    #    return result[result.size // 2:]
    #
    #autocorrelations = np.array([[autocorr(total_array[j])] for j in range(nclasses)]).reshape(nintervals, total_array.shape[1])
    
    stats_dicts = {
        "maxx": maxx,
        "avgg": avgg,
        "meantopkmax":meantopkmax
        #"std_dev": std_devs
    }
    
    print(np.shape(maxx),\
        np.shape(avgg),\
        np.shape(meantopkmax))
    
    return stats_dicts

stats_dic = get_them_stats(p_es_arr_total,labels_total) 

In [ ]:
def generate_colors(num_colors, cmap_name='viridis'):
    cmap = plt.get_cmap(cmap_name)
    colors = [cmap(i) for i in np.linspace(0, 1, num_colors)]
    colors = ['#%02x%02x%02x' % (int(r * 255), int(g * 255), int(b * 255)) for r, g, b, _ in colors]
    return colors


def plot_class_stats(stats, labels_total, anom_labels, anom_labels_i):

    for interval in range(len(stats["maxx"])):
        if labels_total[interval]:
            # Create table data
            header_values = ['Class', 'Max', 'Avg' , 'Mean TopKMax']
            row_values = []

            for i, cls in enumerate(anom_labels_i):
                max_prob_single_value = stats["maxx"][interval][cls]
                avg_prob_single_value = stats["avgg"][interval][cls]
                meantopkmax_single_value = stats["meantopkmax"][interval][cls]
                row_values.append([anom_labels[i], max_prob_single_value, avg_prob_single_value , meantopkmax_single_value])

            # Create table trace
            table_trace = go.Table(
                header=dict(values=header_values, line_color='darkslategray',
                            fill_color='lightskyblue', font=dict(color='white', size=14)),
                cells=dict(values=np.transpose(row_values), line_color='darkslategray',
                        fill_color='lightcyan', font=dict(color='black', size=12))
            )

            # Create figure and add table trace
            fig = go.Figure(data=[table_trace])
            fig.update_layout(title_text=f'Stats @ Interval {interval} with label {labels_total[interval]} {np.shape(p_es_arr_total[interval])}')
            fig.show()
        
        elif interval == 100: break

plot_class_stats(stats_dic, labels_total, CFG_SINET["anom_labels"], CFG_SINET["anom_labels_i"])

In [ ]:
for i in range(len(stats_dic["maxx"])):
    print(i, np.shape(stats_dic[i]))